In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow.keras.utils as ku

In [88]:
df=pd.read_csv(r'C:\Users\Kartikay\Desktop\DS Work\Datasets\demand-forecasting-kernels-only\train.csv')
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [89]:
df.shape

(913000, 4)

In [90]:
df.item.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
      dtype=int64)

In [91]:
df.store.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [92]:
df.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [93]:
df.dtypes

date     object
store     int64
item      int64
sales     int64
dtype: object

In [94]:
df['date']=pd.to_datetime(df['date'])

In [95]:
df.dtypes

date     datetime64[ns]
store             int64
item              int64
sales             int64
dtype: object

In [96]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [97]:
monthly_sales=df.groupby(['date'],as_index=False)['sales'].sum()

In [98]:
monthly_sales

,date,sales
0,2013-01-01,13696
1,2013-01-02,13678
2,2013-01-03,14488
3,2013-01-04,15677
4,2013-01-05,16237
...,...,...
1821,2017-12-27,20378
1822,2017-12-28,21885
1823,2017-12-29,23535
1824,2017-12-30,24988


In [99]:
monthly_sales=pd.DataFrame(monthly_sales.groupby(pd.Grouper(key='date',freq='M'))['sales'].sum())

In [100]:
monthly_sales=monthly_sales.reset_index()

In [101]:
monthly_sales.head(2)

,date,sales
0,2013-01-31,454904
1,2013-02-28,459417


In [102]:
#plt.figure(figsize=[15,6])
#plt.plot(monthly_sales.date,monthly_sales.sales)
#plt.show()
import chart_studio.plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
plot_data = [
    go.Scatter(
        x=monthly_sales['date'],
        y=monthly_sales['sales'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [103]:
plt.style.use('fivethirtyeight')

In [104]:
df_diff=monthly_sales.copy()

In [105]:
df_diff.head()

,date,sales
0,2013-01-31,454904
1,2013-02-28,459417
2,2013-03-31,617382
3,2013-04-30,682274
4,2013-05-31,763242


In [152]:
df_diff['prev_sales']=df_diff['sales'].shift(1)
df_diff.dropna(inplace=True)

df_diff

,date,sales,prev_sales,diff
2,2013-03-31,617382,459417.0,157965.0
3,2013-04-30,682274,617382.0,64892.0
4,2013-05-31,763242,682274.0,80968.0
5,2013-06-30,795597,763242.0,32355.0
6,2013-07-31,855922,795597.0,60325.0
7,2013-08-31,766761,855922.0,-89161.0
8,2013-09-30,689907,766761.0,-76854.0
9,2013-10-31,656587,689907.0,-33320.0
10,2013-11-30,692643,656587.0,36056.0
11,2013-12-31,506607,692643.0,-186036.0


In [107]:
df_diff['diff']=df_diff['sales']-df_diff['prev_sales']

In [108]:
df_diff.head()

,date,sales,prev_sales,diff
1,2013-02-28,459417,454904.0,4513.0
2,2013-03-31,617382,459417.0,157965.0
3,2013-04-30,682274,617382.0,64892.0
4,2013-05-31,763242,682274.0,80968.0
5,2013-06-30,795597,763242.0,32355.0


In [109]:
plot_data = [
    go.Scatter(
        x=df_diff['date'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='Montly Sales Difference'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [110]:
df_supervised=df_diff.drop('prev_sales',axis=1)

In [111]:
df_supervised.head()

,date,sales,diff
1,2013-02-28,459417,4513.0
2,2013-03-31,617382,157965.0
3,2013-04-30,682274,64892.0
4,2013-05-31,763242,80968.0
5,2013-06-30,795597,32355.0


In [112]:
for i in range(1,13):
    field_name='lag_'+ str(i)
    df_supervised[field_name]=df_supervised['diff'].shift(i)

In [113]:
df_supervised=df_supervised.dropna()

In [114]:

df_supervised.head()

,date,sales,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
13,2014-02-28,529117,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0
14,2014-03-31,704301,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0
15,2014-04-30,788914,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0
16,2014-05-31,882877,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0
17,2014-06-30,906842,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0


In [115]:
import statsmodels.formula.api as smf
from sklearn.preprocessing import MinMaxScaler

In [116]:
df_model=df_supervised.drop(['date','sales'],axis=1)

In [118]:
df_model


,diff,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,lag_10,lag_11,lag_12
13,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0,4513.0
14,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0,157965.0
15,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0,64892.0
16,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0,80968.0
17,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0,32355.0
18,82168.0,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0,60325.0
19,-103414.0,82168.0,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0,-89161.0
20,-100472.0,-103414.0,82168.0,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0,-76854.0
21,-26241.0,-100472.0,-103414.0,82168.0,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0,-33320.0
22,41900.0,-26241.0,-100472.0,-103414.0,82168.0,23965.0,93963.0,84613.0,175184.0,3130.0,19380.0,-186036.0,36056.0


In [83]:
train_set,test_set=df_model[0:-6].values,df_model[-6:].values

In [84]:
train_set

array([[   3130.,   19380., -186036.,   36056.,  -33320.,  -76854.,
         -89161.,   60325.,   32355.,   80968.,   64892.,  157965.,
           4513.],
       [ 175184.,    3130.,   19380., -186036.,   36056.,  -33320.,
         -76854.,  -89161.,   60325.,   32355.,   80968.,   64892.,
         157965.],
       [  84613.,  175184.,    3130.,   19380., -186036.,   36056.,
         -33320.,  -76854.,  -89161.,   60325.,   32355.,   80968.,
          64892.],
       [  93963.,   84613.,  175184.,    3130.,   19380., -186036.,
          36056.,  -33320.,  -76854.,  -89161.,   60325.,   32355.,
          80968.],
       [  23965.,   93963.,   84613.,  175184.,    3130.,   19380.,
        -186036.,   36056.,  -33320.,  -76854.,  -89161.,   60325.,
          32355.],
       [  82168.,   23965.,   93963.,   84613.,  175184.,    3130.,
          19380., -186036.,   36056.,  -33320.,  -76854.,  -89161.,
          60325.],
       [-103414.,   82168.,   23965.,   93963.,   84613.,  175184.,
  

In [85]:
test_set

array([[ 106769.,   43938.,   81824.,  116195.,  201298.,    4063.,
         -46105., -228037.,   27811.,  -33194.,  -84663., -157224.,
         116054.],
       [-144990.,  106769.,   43938.,   81824.,  116195.,  201298.,
           4063.,  -46105., -228037.,   27811.,  -33194.,  -84663.,
        -157224.],
       [ -91140., -144990.,  106769.,   43938.,   81824.,  116195.,
         201298.,    4063.,  -46105., -228037.,   27811.,  -33194.,
         -84663.],
       [ -44103.,  -91140., -144990.,  106769.,   43938.,   81824.,
         116195.,  201298.,    4063.,  -46105., -228037.,   27811.,
         -33194.],
       [  37677.,  -44103.,  -91140., -144990.,  106769.,   43938.,
          81824.,  116195.,  201298.,    4063.,  -46105., -228037.,
          27811.],
       [-233667.,   37677.,  -44103.,  -91140., -144990.,  106769.,
          43938.,   81824.,  116195.,  201298.,    4063.,  -46105.,
        -228037.]])

In [119]:
scaler=MinMaxScaler(feature_range=(-1, 1))
scaler.fit(train_set)

MinMaxScaler(feature_range=(-1, 1))

In [120]:
train_set = train_set.reshape(train_set.shape[0], train_set.shape[1])
train_set_scaled=scaler.transform(train_set)

In [121]:
train_set_scaled

array([[ 0.07686073,  0.15255919, -0.80434393,  0.23024212, -0.0447346 ,
        -0.25830878, -0.3186859 ,  0.40696724,  0.26794062,  0.50957454,
         0.42966779,  0.8922929 ,  0.12955024],
       [ 0.8783514 ,  0.07686073,  0.15255919, -0.80434393,  0.29561828,
        -0.0447346 , -0.25830878, -0.33606217,  0.40696724,  0.26794062,
         0.50957454,  0.42966779,  0.8922929 ],
       [ 0.45643845,  0.8783514 ,  0.07686073,  0.15255919, -0.79394659,
         0.29561828, -0.0447346 , -0.27488947, -0.33606217,  0.40696724,
         0.26794062,  0.50957454,  0.42966779],
       [ 0.49999418,  0.45643845,  0.8783514 ,  0.07686073,  0.21380721,
        -0.79394659,  0.29561828, -0.05850103, -0.27488947, -0.33606217,
         0.40696724,  0.26794062,  0.50957454],
       [ 0.1739178 ,  0.49999418,  0.45643845,  0.8783514 ,  0.13408606,
         0.21380721, -0.79394659,  0.28633667, -0.05850103, -0.27488947,
        -0.33606217,  0.40696724,  0.26794062],
       [ 0.44504874,  0.173917

In [122]:
test_set = test_set.reshape(test_set.shape[0], test_set.shape[1])
test_set_scaled = scaler.transform(test_set)

In [124]:
test_set

array([[ 106769.,   43938.,   81824.,  116195.,  201298.,    4063.,
         -46105., -228037.,   27811.,  -33194.,  -84663., -157224.,
         116054.],
       [-144990.,  106769.,   43938.,   81824.,  116195.,  201298.,
           4063.,  -46105., -228037.,   27811.,  -33194.,  -84663.,
        -157224.],
       [ -91140., -144990.,  106769.,   43938.,   81824.,  116195.,
         201298.,    4063.,  -46105., -228037.,   27811.,  -33194.,
         -84663.],
       [ -44103.,  -91140., -144990.,  106769.,   43938.,   81824.,
         116195.,  201298.,    4063.,  -46105., -228037.,   27811.,
         -33194.],
       [  37677.,  -44103.,  -91140., -144990.,  106769.,   43938.,
          81824.,  116195.,  201298.,    4063.,  -46105., -228037.,
          27811.],
       [-233667.,   37677.,  -44103.,  -91140., -144990.,  106769.,
          43938.,   81824.,  116195.,  201298.,    4063.,  -46105.,
        -228037.]])

In [125]:
X_train, y_train = train_set_scaled[:, 1:], train_set_scaled[:, 0:1]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test, y_test = test_set_scaled[:, 1:], test_set_scaled[:, 0:1]
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [126]:
X_train

array([[[ 0.15255919, -0.80434393,  0.23024212, -0.0447346 ,
         -0.25830878, -0.3186859 ,  0.40696724,  0.26794062,
          0.50957454,  0.42966779,  0.8922929 ,  0.12955024]],

       [[ 0.07686073,  0.15255919, -0.80434393,  0.29561828,
         -0.0447346 , -0.25830878, -0.33606217,  0.40696724,
          0.26794062,  0.50957454,  0.42966779,  0.8922929 ]],

       [[ 0.8783514 ,  0.07686073,  0.15255919, -0.79394659,
          0.29561828, -0.0447346 , -0.27488947, -0.33606217,
          0.40696724,  0.26794062,  0.50957454,  0.42966779]],

       [[ 0.45643845,  0.8783514 ,  0.07686073,  0.21380721,
         -0.79394659,  0.29561828, -0.05850103, -0.27488947,
         -0.33606217,  0.40696724,  0.26794062,  0.50957454]],

       [[ 0.49999418,  0.45643845,  0.8783514 ,  0.13408606,
          0.21380721, -0.79394659,  0.28633667, -0.05850103,
         -0.27488947, -0.33606217,  0.40696724,  0.26794062]],

       [[ 0.1739178 ,  0.49999418,  0.45643845,  0.97816867,
         

In [127]:
y_train

array([[ 0.07686073],
       [ 0.8783514 ],
       [ 0.45643845],
       [ 0.49999418],
       [ 0.1739178 ],
       [ 0.44504874],
       [-0.41946033],
       [-0.40575541],
       [-0.05996017],
       [ 0.25746562],
       [-0.97530134],
       [-0.05667136],
       [ 0.05670863],
       [ 0.89908114],
       [ 0.49791189],
       [ 0.53945986],
       [ 0.11017737],
       [ 0.52889003],
       [-0.48251133],
       [-0.38990299],
       [-0.05920552],
       [ 0.20385713],
       [-0.96290077],
       [ 0.03836864],
       [ 0.12059348],
       [ 0.8817986 ],
       [ 0.5796802 ],
       [ 0.46653313],
       [ 0.22035706],
       [ 0.60290216],
       [-0.67012706],
       [-0.33211129],
       [-0.0923498 ],
       [ 0.19183388],
       [-1.        ],
       [-0.15249397],
       [ 0.08120698],
       [ 1.        ],
       [ 0.60355899],
       [ 0.44344626],
       [ 0.26695937]])

In [129]:
X_test

array([[[ 0.26695937,  0.44344626,  0.60355899,  1.10628178,
          0.13866328, -0.10745675, -1.02635392,  0.24535439,
         -0.05787474, -0.31370458, -0.67437352,  0.68397168]],

       [[ 0.55964922,  0.26695937,  0.44344626,  0.68877355,
          1.10628178,  0.13866328, -0.12204966, -1.02635392,
          0.24535439, -0.05787474, -0.31370458, -0.67437352]],

       [[-0.61313659,  0.55964922,  0.26695937,  0.52015228,
          0.68877355,  1.10628178,  0.12731349, -0.12204966,
         -1.02635392,  0.24535439, -0.05787474, -0.31370458]],

       [[-0.36228353, -0.61313659,  0.55964922,  0.33428672,
          0.52015228,  0.68877355,  1.10768225,  0.12731349,
         -0.12204966, -1.02635392,  0.24535439, -0.05787474]],

       [[-0.14316792, -0.36228353, -0.61313659,  0.64253037,
          0.33428672,  0.52015228,  0.68467253,  1.10768225,
          0.12731349, -0.12204966, -1.02635392,  0.24535439]],

       [[ 0.23779333, -0.14316792, -0.36228353, -0.59257833,
         

In [130]:
y_test

array([[ 0.55964922],
       [-0.61313659],
       [-0.36228353],
       [-0.14316792],
       [ 0.23779333],
       [-1.02622661]])

In [131]:
model=Sequential()
model.add(LSTM(100, batch_input_shape=(1, X_train.shape[1], X_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(optimizer='Adam',loss='mean_squared_error')
model.fit(X_train,y_train,nb_epoch=100,batch_size=1, verbose=1, shuffle=False)

Train on 41 samples
Epoch 1/100
41/41 [==============================] - 3s 82ms/sample - loss: 0.1501
Epoch 2/100
41/41 [==============================] - 0s 4ms/sample - loss: 0.0810
Epoch 3/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0463
Epoch 4/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0243
Epoch 5/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0145
Epoch 6/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0110
Epoch 7/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0095
Epoch 8/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0089
Epoch 9/100
41/41 [==============================] - 0s 4ms/sample - loss: 0.0085
Epoch 10/100
41/41 [==============================] - 0s 4ms/sample - loss: 0.0082
Epoch 11/100
41/41 [==============================] - 0s 3ms/sample - loss: 0.0079
Epoch 12/100
41/41 [==============================] - 0s 3ms/sample - loss:

In [132]:
y_pred = model.predict(X_test,batch_size=1)

In [133]:
y_pred

array([[ 0.6704461 ],
       [-0.78572017],
       [-0.29254377],
       [-0.11287521],
       [ 0.1708381 ],
       [-1.0077392 ]], dtype=float32)

In [134]:
y_test

array([[ 0.55964922],
       [-0.61313659],
       [-0.36228353],
       [-0.14316792],
       [ 0.23779333],
       [-1.02622661]])

In [136]:
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])

In [143]:
X_test.shape

(6, 1, 12)

In [137]:
y_pred.shape

(6, 1, 1)

In [138]:
import numpy as np

In [139]:
len(y_pred)

6

In [140]:
pred_test_set = []
for index in range(0,len(y_pred)):
    print (np.concatenate([y_pred[index],X_test[index]],axis=1))
    pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

[[ 0.6704461   0.26695937  0.44344626  0.60355899  1.10628178  0.13866328
  -0.10745675 -1.02635392  0.24535439 -0.05787474 -0.31370458 -0.67437352
   0.68397168]]
[[-0.78572017  0.55964922  0.26695937  0.44344626  0.68877355  1.10628178
   0.13866328 -0.12204966 -1.02635392  0.24535439 -0.05787474 -0.31370458
  -0.67437352]]
[[-0.29254377 -0.61313659  0.55964922  0.26695937  0.52015228  0.68877355
   1.10628178  0.12731349 -0.12204966 -1.02635392  0.24535439 -0.05787474
  -0.31370458]]
[[-0.11287521 -0.36228353 -0.61313659  0.55964922  0.33428672  0.52015228
   0.68877355  1.10768225  0.12731349 -0.12204966 -1.02635392  0.24535439
  -0.05787474]]
[[ 0.1708381  -0.14316792 -0.36228353 -0.61313659  0.64253037  0.33428672
   0.52015228  0.68467253  1.10768225  0.12731349 -0.12204966 -1.02635392
   0.24535439]]
[[-1.00773919  0.23779333 -0.14316792 -0.36228353 -0.59257833  0.64253037
   0.33428672  0.51382935  0.68467253  1.10768225  0.12731349 -0.12204966
  -1.02635392]]


In [52]:
len(pred_test_set)

6

In [53]:
#pred_test_set.append(np.concatenate([y_pred[index],X_test[index]],axis=1))

In [54]:
pred_test_set

[array([[ 0.78089702,  0.63347968,  0.72172313,  0.8017795 ,  1.05314089,
          0.56933164,  0.44627163, -0.01317696,  0.62267719,  0.47106263,
          0.34314771,  0.16281324,  0.84198584]]),
 array([[ 0.08644949,  0.77982461,  0.63347968,  0.72172313,  0.84438677,
          1.05314089,  0.56933164,  0.43897517, -0.01317696,  0.62267719,
          0.47106263,  0.34314771,  0.16281324]]),
 array([[ 0.31946462,  0.1934317 ,  0.77982461,  0.63347968,  0.76007614,
          0.84438677,  1.05314089,  0.56365674,  0.43897517, -0.01317696,
          0.62267719,  0.47106263,  0.34314771]]),
 array([[ 0.41652367,  0.31885823,  0.1934317 ,  0.77982461,  0.66714336,
          0.76007614,  0.84438677,  1.05384113,  0.56365674,  0.43897517,
         -0.01317696,  0.62267719,  0.47106263]]),
 array([[ 0.56396157,  0.42841604,  0.31885823,  0.1934317 ,  0.82126519,
          0.66714336,  0.76007614,  0.84233626,  1.05384113,  0.56365674,
          0.43897517, -0.01317696,  0.62267719]]),
 arra

In [144]:
pred_test_set = np.array(pred_test_set)
pred_test_set = pred_test_set.reshape(pred_test_set.shape[0], pred_test_set.shape[2])

In [145]:
pred_test_set.shape

(6, 13)

In [148]:
pred_test_set_inverted = scaler.inverse_transform(pred_test_set)
pred_test_set_inverted

array([[ 130553.48771039,   43938.        ,   81824.        ,
         116195.        ,  201298.        ,    4063.        ,
         -46105.        , -228037.        ,   27811.        ,
         -33194.        ,  -84663.        , -157224.        ,
         116054.        ],
       [-182038.08449563,  106769.        ,   43938.        ,
          81824.        ,  116195.        ,  201298.        ,
           4063.        ,  -46105.        , -228037.        ,
          27811.        ,  -33194.        ,  -84663.        ,
        -157224.        ],
       [ -76169.13950664, -144990.        ,  106769.        ,
          43938.        ,   81824.        ,  116195.        ,
         201298.        ,    4063.        ,  -46105.        ,
        -228037.        ,   27811.        ,  -33194.        ,
         -84663.        ],
       [ -37600.13876886,  -91140.        , -144990.        ,
         106769.        ,   43938.        ,   81824.        ,
         116195.        ,  201298.        ,    4063

In [150]:
pred_test_set_inverted[1][0]

-182038.08449563384

In [151]:
act_sales = list(df[-6:].sales)
act_sales

[41, 63, 59, 74, 62, 82]

In [154]:
#create dataframe that shows the predicted sales
result_list = []
sales_dates = list(df_supervised[-7:].date)
act_sales = list(df_supervised[-7:].sales)
for index in range(0,len(pred_test_set_inverted)):
    result_dict = {}
    result_dict['pred_value'] = int(pred_test_set_inverted[index][0] + act_sales[index])
    result_dict['date'] = sales_dates[index+1]
    result_list.append(result_dict)
df_result = pd.DataFrame(result_list)
#for multistep prediction, replace act_sales with the predicted sales

In [155]:
sales_dates

[Timestamp('2017-06-30 00:00:00'),
 Timestamp('2017-07-31 00:00:00'),
 Timestamp('2017-08-31 00:00:00'),
 Timestamp('2017-09-30 00:00:00'),
 Timestamp('2017-10-31 00:00:00'),
 Timestamp('2017-11-30 00:00:00'),
 Timestamp('2017-12-31 00:00:00')]

In [156]:
act_sales

[1064624, 1171393, 1026403, 935263, 891160, 928837, 695170]

In [157]:
result_list

[{'pred_value': 1195177, 'date': Timestamp('2017-07-31 00:00:00')},
 {'pred_value': 989354, 'date': Timestamp('2017-08-31 00:00:00')},
 {'pred_value': 950233, 'date': Timestamp('2017-09-30 00:00:00')},
 {'pred_value': 897662, 'date': Timestamp('2017-10-31 00:00:00')},
 {'pred_value': 914463, 'date': Timestamp('2017-11-30 00:00:00')},
 {'pred_value': 699138, 'date': Timestamp('2017-12-31 00:00:00')}]

In [158]:
df_result

,pred_value,date
0,1195177,2017-07-31
1,989354,2017-08-31
2,950233,2017-09-30
3,897662,2017-10-31
4,914463,2017-11-30
5,699138,2017-12-31


In [161]:
df_sales_pred = pd.merge(df_supervised,df_result,on='date',how='left')

In [162]:
#plot actual and predicted
plot_data = [
    go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['sales'],
        name='actual'
    ),
        go.Scatter(
        x=df_sales_pred['date'],
        y=df_sales_pred['pred_value'],
        name='predicted'
    )
    
]
plot_layout = go.Layout(
        title='Sales Prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)